In [1]:
# Importing dependencies 
import pandas as pd
import numpy as np
import scipy.stats as st
import os

# Importing  both time traveled to work csvs and concatonating into a sigle dataframe
time_traveled_dir = ['Travel Time to Work', 'Travel Time County and State Level']
for dir in time_traveled_dir:

    path = f'Resources/{dir}'
    files = os.listdir(path)

    travel_df = pd.DataFrame()
    cs_travel_df = pd.DataFrame()

    for file in files:
        if dir == 'Travel Time to Work':
            temp = pd.read_csv(path + '\\' + file, skiprows=[1])
            temp = temp[['B08303_001E', 'B08303_002E', 'B08303_003E', 'B08303_004E', 'B08303_005E', 'B08303_006E', 'B08303_007E', \
                'B08303_008E', 'B08303_009E', 'B08303_010E', 'B08303_011E', 'B08303_012E', 'B08303_013E', 'NAME']]
            # Renaming columns 
            temp.columns = ['Total Estimate', 'Estimate Less than 5 min', 'Estimate 5 to 9 min', 'Estimate 10 to 14 min', \
                'Estimate 15 to 19 min', 'Estimate 20 to 24 min', 'Estimate 25 to 29 min', 'Estimate 30 to 34 min', 'Estimate 35 to 39 min', \
                'Estimate 40 to 44 min', 'Estimate 45 to 59 min', 'Estimate 60 to 89 min', 'Estimate 90 or more min', 'Name']
            # Adding a column for respective years
            temp['Year'] = file.split(' ')[0]
            # Reordering columns
            temp = temp[['Year', 'Name', 'Total Estimate', 'Estimate Less than 5 min', 'Estimate 5 to 9 min', 'Estimate 10 to 14 min', \
                'Estimate 15 to 19 min', 'Estimate 20 to 24 min', 'Estimate 25 to 29 min', 'Estimate 30 to 34 min', 'Estimate 35 to 39 min', \
                'Estimate 40 to 44 min', 'Estimate 45 to 59 min', 'Estimate 60 to 89 min', 'Estimate 90 or more min']]
            # Appending each new temporary dataframe into one
            travel_df = travel_df.append(temp)
        elif dir == 'Travel Time County and State Level':
            temp = pd.read_csv(path + '\\' + file, skiprows=[1])
            temp = temp[['B08131_001E', 'B08131_002E', 'B08131_003E', 'B08131_004E', 'B08131_005E', 'NAME']]
            # Renaming columns 
            temp.columns = ['Estiamte Aggregate Travel Time (min)', 'Worked in State of Residence Estimate Aggregate (min)', \
                'Worked in County of Residence Estimate Aggregate (min)', 'Worked Outside County of Residence Estimate Aggregate (min)', \
                'Worked Outside State of Residence Estimate Aggregate (min)', 'Name']
            # Adding a column for respective years
            temp['Year'] = file.split(' ')[0]
            # Reordering columns
            temp = temp[['Year', 'Name', 'Estiamte Aggregate Travel Time (min)', 'Worked in State of Residence Estimate Aggregate (min)', \
                'Worked in County of Residence Estimate Aggregate (min)', 'Worked Outside County of Residence Estimate Aggregate (min)', \
                'Worked Outside State of Residence Estimate Aggregate (min)']]
            # Appending each new temporary dataframe into one
            cs_travel_df = cs_travel_df.append(temp)

# Importing HPI csv and making into a dataframe
hpi_quarterly = pd.read_csv('Resources/HPI/HPI Quarterly State Level.csv')
# Removing Warning column
hpi_quarterly = hpi_quarterly[['state', 'yr', 'qtr', 'index_nsa', 'index_sa']]
# Renaming columns
hpi_quarterly.columns = ['State', 'Year', 'Quarter', 'NSA Index', 'SA Index']
hpi_quarterly_df = pd.DataFrame(hpi_quarterly)

In [2]:
hpi_quarterly_df

,State,Year,Quarter,NSA Index,SA Index
0,AK,1991,1,100.00,100.00
1,AK,1991,2,101.01,100.25
2,AK,1991,3,102.03,101.23
3,AK,1991,4,102.42,102.27
4,AK,1992,1,102.80,102.74
...,...,...,...,...,...
6319,WY,2020,4,395.46,392.51
6320,WY,2021,1,403.28,404.45
6321,WY,2021,2,428.13,421.78
6322,WY,2021,3,441.94,436.97


# HPI Summary

# Time Traveled to Work Summary